In [2]:
import json
import requests

In [4]:
#Prosta serializacja
data = {
    "student": {
        "name": "Piotr Kowalecki",
        "group": "175IC_B1"
    }
}

#Zapisanie do pliku data_file o rozszerzeniu JSON
with open("json_file/data_file.json", "w") as write_file:
  json.dump(data, write_file)


json.dumps(data)

'{"student": {"name": "Piotr Kowalecki", "group": "175IC_B1"}}'

In [5]:
#Dodanie wycięć w wyświetlaniu danych
json.dumps(data, indent=3)

'{\n   "student": {\n      "name": "Piotr Kowalecki",\n      "group": "175IC_B1"\n   }\n}'

In [9]:
#Deserializacja z pliku
with open("json_file/data_file_deserialization.json", "r") as read_file:
    data = json.load(read_file)
data

{'student': {'group': '175IC_B1',
  'name': 'Piotr Kowalecki',
  'type': 'deserialization'}}

In [26]:
#Deserializacja z URL
response = requests.get("https://jsonplaceholder.typicode.com/posts")
posts = json.loads(response.text)

In [38]:
#Wyświetlanie ukończonych zadań
posts_by_user = {}

for post in posts:
  if post["id"]:
    try:
      posts_by_user[post["userId"]] += 1
    except KeyError:
      posts_by_user[post["userId"]] = 1

top_posts = sorted(posts_by_user.items(), 
                   key=lambda x: x[1], reverse=True)

print(f'Top postów: {top_posts}')

max_complete = top_posts[0][1]
users = []
for user, num_complete in top_posts:
    if num_complete < max_complete:
        break
    users.append(str(user))

max_users = ", ".join(users)

print(f'ID osób które mają najwięcej postów: {max_users}')

Top postów: [(1, 10), (2, 10), (3, 10), (4, 10), (5, 10), (6, 10), (7, 10), (8, 10), (9, 10), (10, 10)]
ID osób które mają najwięcej postów: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10


In [ ]:
#Funkcja wypisująca posty osób, które mają ich najwięcej (niestety api ma podzielone po równo)
def keep(post):
    is_complete = post["id"]
    has_max_count = str(post["userId"]) in users
    return is_complete and has_max_count

#Zapis danych do JSON
with open("json_file/filtered_data_file.json", "w") as data_file:
    filtered_posts = list(filter(keep, posts))
    json.dump(filtered_posts, data_file, indent=2)

filtered_posts

In [46]:
#Kodowanie i dekodowanie niestandardowych obiektów Pythona
def encode_complex(z):
    if isinstance(z, complex):
        return (z.real, z.imag)
    else:
        type_name = z.__class__.__name__
        raise TypeError(f"Object of type '{type_name}' is not JSON serializable")
json.dumps(9 + 5j, default=encode_complex)

'[9.0, 5.0]'

In [47]:
#Kodowanie z pomocą JSONEncoder
class ComplexEncoder(json.JSONEncoder):
    def default(self, z):
        if isinstance(z, complex):
            return (z.real, z.imag)
        else:
            return super().default(z)

#Przykładowe wywołania kodowania powyżej
print(f'json.dumps: {json.dumps(2 + 6j, cls=ComplexEncoder)}')
encoder = ComplexEncoder()
print(f'encoder.encode: {encoder.encode(3 + 6j)}')

#Dekodowanie typów niestandardowych
complex_json = json.dumps(4 + 17j, cls=ComplexEncoder)
print(f'json.loads: {json.loads(complex_json)}')

json.dumps: [2.0, 6.0]
encoder.encode: [3.0, 6.0]
json.loads: [4.0, 17.0]


In [49]:
#Dekodowanie klucza - definicja oraz odkodowanie
def decode_complex(dct):
    if "__complex__" in dct:
        return complex(dct["real"], dct["imag"])
    return dct

with open("json_file/complex_data.json") as complex_data:
     data = complex_data.read()
     z = json.loads(data, object_hook=decode_complex)

print(z)

[(42+36j), (64+11j), 0j]
